# Desafio de Dados:
<img src =  "assets/photo_2023-10-25_21-25-10.jpg">

**Teremos 5 camadas em nossa estrutura, cada camada será representada por uma pasta dentro do bucket.**

**Bucket D2D**

<span style="font-weight: bold;">

- Transient -1
- Raw -2
- Trusted -3
- Refined -4
- Sandbox
  - Transient
  - Raw
  - Trusted
  - Refined
  
</span>
  
# CONSTRUINDO A SOLUÇÃO:


In [1]:
# Importa as bibliotecas necessárias
print("Importando as bibliotecas necessárias")
import os
from pyspark import SparkConf
from pyspark.sql import SparkSession

Importando as bibliotecas necessárias


In [2]:
# Configura a variável de ambiente PYSPARK_SUBMIT_ARGS
print("Configurando a variável de ambiente PYSPARK_SUBMIT_ARGS")
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.hadoop:hadoop-aws:3.3.1 pyspark-shell'

# Cria uma sessão Spark
print("Iniciando a sessão Spark")
spark = SparkSession.builder.appName("ExemploMinIO").getOrCreate()

# Configura as configurações do MinIO (caso não estejam definidas em PYSPARK_SUBMIT_ARGS)
print("Configurando as configurações do MinIO")
spark._jsc.hadoopConfiguration().set("fs.s3a.access.key", "qiBoC6oxvaAdlFCR9I9p")
spark._jsc.hadoopConfiguration().set("fs.s3a.secret.key", "vjHjYMCp6fbWyHjO3iFHewSggT0lWiCBFNGVKiu2")
spark._jsc.hadoopConfiguration().set("fs.s3a.endpoint", "http://172.17.59.191:9000")

Configurando a variável de ambiente PYSPARK_SUBMIT_ARGS
Iniciando a sessão Spark
Configurando as configurações do MinIO


In [3]:
print("Definindo variáveis")
minio_bucket_name = "d2d"
minio_path = "transient/arquivo_teste_1"  # O caminho no MinIO onde será salvo

# Cria um DataFrame
print("Criando um DataFrame")
df = spark.createDataFrame([(1, "Lucas", 28), (2, "Alana", 22), 
                            (3, "Gabrielly", 45), (4, "Isabela", 4), 
                            (5, "Jhonny",1), (6, "Fernando", 60)], 
                           ["id", "nome", "idade"])

# Salva o DataFrame diretamente no MinIO
print("Salvando o DataFrame diretamente no MinIO")
df.write.format("parquet") \
    .mode("overwrite") \
    .option("path", f"s3a://{minio_bucket_name}/{minio_path}") \
    .save()
print("DataFrame no bucket do MinIO")

# Encerra a sessão Spark
#spark.stop()

Definindo variáveis
Criando um DataFrame
Salvando o DataFrame diretamente no MinIO
DataFrame no bucket do MinIO


In [4]:
#ler o arquivo salvo
arq = spark.read.parquet(f"s3a://{minio_bucket_name}/{minio_path}")
arq.show()

+---+---------+-----+
| id|     nome|idade|
+---+---------+-----+
|  3|Gabrielly|   45|
|  6| Fernando|   60|
|  4|  Isabela|    4|
|  5|   Jhonny|    1|
|  1|    Lucas|   28|
|  2|    Alana|   22|
+---+---------+-----+



In [5]:
# Encerra a sessão Spark
print("Sessão spark encerrada")
spark.stop()

Sessão spark encerrada
